In [1]:
""" Apply the combined mask and calculate the max discharge per 30spfaf06 zone.
-------------------------------------------------------------------------------
Update: make the script server side.

The combined mask is composed of two components: 1) Subbasins need to be
sufficiently large and 2) the number of maximum streamorder cells needs to be
sufficient. Thresholds as of 20180528.

Area > 1000 cells (30s)
Streamorder > 150 cells (30s)

The combined mask is applied to the zones and a zonal statistic (max) is 
calculted with volumetric riverdischarge as input.

The output will be stored as table. Options include: 
1) dataframe on EC2
2) CSV file on GCS
3) fc on ee

depending on performance, we will choose option 1,2 or 3.

Author: Rutger Hofste
Date: 20180528
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    TESTING (boolean) : Testing mode. Uses a smaller geography if enabled.
    
    SCRIPT_NAME (string) : Script name.
    EE_INPUT_ZONES_PATH (string) : earthengine input path for zones.
    EE_INPUT_VALUES_PATH (string) : earthengine input path for value images.
    INPUT_VERSION_ZONES (integer) : input version for zones images.
    INPUT_VERSION_VALUES (integer) : input version for value images.
    OUTPUT_VERSION (integer) : output version. 
    EXTRA_PROPERTIES (dictionary) : Extra properties to store in the resulting
        pandas dataframe. 
    

Returns:

"""

TESTING = 0
OVERWRITE = 1
SCRIPT_NAME = "Y2018M05D28_RH_Riverdischarge_Mainchannel_30sPfaf06_EE_V02"
OUTPUT_VERSION = 4

EE_INPUT_ASSET_ID_30SPFAF06ZONES = "projects/WRI-Aquaduct/Y2018M04D20_RH_Ingest_HydroBasins_GCS_EE_V01/output_V02/hybas_lev06_v1c_merged_fiona_30s_V04"
EE_INPUT_ASSET_ID_COMBINEDMASK = "projects/WRI-Aquaduct/Y2018M05D03_RH_Mask_Discharge_Pixels_V01/output_V04/global_riverdischarge_mask_30sPfaf06"
EE_INPUT_RIVERDISCHARGE_PATH_ID = "projects/WRI-Aquaduct/PCRGlobWB20V09/"

GCS_BUCKET = "aqueduct30_v01"
GCS_OUTPUT_PATH = "{}/outputV{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

SEPARATOR = "_|-"
SCHEMA =["geographic_range",
         "temporal_range",
         "indicator",
         "temporal_resolution",
         "unit",
         "spatial_resolution",
         "temporal_range_min",
         "temporal_range_max"]

EXTRA_PROPERTIES = {"output_version":OUTPUT_VERSION,
                    "script_used":SCRIPT_NAME,
                   }

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)


print("Input ee zones: " +  EE_INPUT_ASSET_ID_30SPFAF06ZONES +
      "\nInput ee mask: " + EE_INPUT_ASSET_ID_COMBINEDMASK  +
      "\nInput ee riverdischarge month: " + EE_INPUT_RIVERDISCHARGE_PATH_ID,
      "\nOutput ec2: " + ec2_output_path,
      "\nOutput s3: " + s3_output_path)




Input ee zones: projects/WRI-Aquaduct/Y2018M04D20_RH_Ingest_HydroBasins_GCS_EE_V01/output_V02/hybas_lev06_v1c_merged_fiona_30s_V04
Input ee mask: projects/WRI-Aquaduct/Y2018M05D03_RH_Mask_Discharge_Pixels_V01/output_V04/global_riverdischarge_mask_30sPfaf06
Input ee riverdischarge month: projects/WRI-Aquaduct/PCRGlobWB20V09/ 
Output ec2: /volumes/data/Y2018M05D28_RH_Riverdischarge_Mainchannel_30sPfaf06_EE_V02/output_V04 
Output s3: s3://wri-projects/Aqueduct30/processData/Y2018M05D28_RH_Riverdischarge_Mainchannel_30sPfaf06_EE_V02/output_V04


In [2]:
import time, datetime, sys, logging
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M05D29 UTC 09:59


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
file_handler = logging.FileHandler("./logs/{}.log".format(SCRIPT_NAME))
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [4]:
# Imports
import pandas as pd
import numpy as np
from datetime import timedelta
import os
import ee
import aqueduct3

ee.Initialize()

In [5]:
def post_process_results(result_list,function_properties,extra_properties=EXTRA_PROPERTIES):
    """Client side function to convert results of reduceRegion to pandas dataframe.
    -------------------------------------------------------------------------------
    
    Adds additional properties. The script is client side for convenience reasons.
    A more robust and fast approach would be to add the extra_properties to the 
    server side dictionary.
    
    Args:
        result_list (ee.List) : List of dictionaries. Result from reduceRegion
        function_properties (dictionary) : Additional properties used in the 
            reduceRegion function call.
        extra_properties (dictionary) : Additional properties set at global level. 
    
    Returns:
        df (pd.DataFrame) : Pandas dataframe with extra properties.
    
    
    """
    extra_properties = {**function_properties, **EXTRA_PROPERTIES}
    result_list_clientside = result_list.getInfo()
    df = pd.DataFrame(result_list_clientside)
    df = df.assign(**extra_properties)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df  

In [6]:
# helper functions:

def dict_to_feature(d):
    """ Convert a dictionary to an earth engine feature
    server side
    
    Args:
        d (ee.Dictionary) : server side dictionary
    Returns
        f (ee.Feature) : server side earth engine feature    
    """
    f = ee.Feature(None,ee.Dictionary(d))
    return f

In [7]:
def add_extra_properties(feature):
    return ee.Feature(feature).setMulti(extra_properties)



def post_process_results_server_side(result_list,function_properties,extra_properties=EXTRA_PROPERTIES):
    """ Convert list of dicts to featureCollection, Add properties and export as 
    CSV
    -------------------------------------------------------------------------------
    
    Args:
        result_list (ee.List) : List of dictionaries. Result from reduceRegion
        function_properties (dictionary) : Additional properties used in the 
            reduceRegion function call.
        extra_properties (dictionary) : Additional properties set at global level.   
        
        
    TODO: Assess how bad the client side function performs. Convert List to FC, add properties, save as CSV. 
    
    """
    
    extra_properties = {**function_properties, **EXTRA_PROPERTIES}
    fc = ee.FeatureCollection(result_list.map(dict_to_feature))
    fc_out = fc.map(lambda x: ee.Feature(x).setMulti(extra_properties) )
    
    return fc_out
    
    
    

In [8]:
i_zones = ee.Image(EE_INPUT_ASSET_ID_30SPFAF06ZONES)
i_combined_mask = ee.Image(EE_INPUT_ASSET_ID_COMBINEDMASK)

In [9]:
temporal_resolutions = ["month","year"]
spatial_resolution = "30s"
pfaf_level = 6
indicator = "riverdischarge"
reducer_name = "max"

if TESTING:
    temporal_resolution = ["month"]

In [10]:
# Apply mask
i_maskedzones_30sPfaf06 = i_zones.mask(i_combined_mask)

# Geospatial constants
geometry_server_side = aqueduct3.earthengine.get_global_geometry(test=TESTING)
geometry_client_side = geometry_server_side.getInfo()['coordinates']

crs_transform = aqueduct3.earthengine.get_crs_transform(spatial_resolution)


In [11]:
i_processed = 0
start_time = time.time()


for temporal_resolution in temporal_resolutions:
    ic_values_input_asset_id = "{}global_historical_{}_{}_millionm3_5min_1960_2014".format(EE_INPUT_RIVERDISCHARGE_PATH_ID,indicator,temporal_resolution)
    print(ic_values_input_asset_id)
    df = aqueduct3.earthengine.get_df_from_ic(ic_values_input_asset_id)
    if TESTING:
        df = df[0:3]
    else:
        pass
    
    for index, row in df.iterrows():
        i_processed = i_processed + 1
        elapsed_time = time.time() - start_time
        i_values_input_asset_id = row["input_image_asset_id"]
        # Add an artificial extension to allow the function to run. 
        # consider updating the split_key function to handle cases without an extension.
        i_values_input_asset_id_extenstion = i_values_input_asset_id + ".ee_image"
        dictje = aqueduct3.split_key(i_values_input_asset_id_extenstion,SCHEMA,SEPARATOR)
                
        message = "Index {:02.2f}, Processed: {} Elapsed: {} Asset: {}".format(float(index),i_processed,str(timedelta(seconds=elapsed_time)),i_values_input_asset_id)
        print(message)
        logger.debug(message)

        i_values = ee.Image(i_values_input_asset_id)

        result_list = aqueduct3.earthengine.raster_zonal_stats(
                                    i_zones = i_maskedzones_30sPfaf06,
                                    i_values = i_values,
                                    statistic_type = reducer_name,
                                    geometry = geometry_server_side,
                                    crs_transform = crs_transform,
                                    crs="EPSG:4326")


        function_properties = {"zones_pfaf_level":pfaf_level,
                               "zones_spatial_resolution":spatial_resolution,
                               "reducer":reducer_name,
                               "zones_image_asset_id":EE_INPUT_ASSET_ID_30SPFAF06ZONES,
                               "mask_image_asset_id" :EE_INPUT_ASSET_ID_COMBINEDMASK}

        function_properties = {**function_properties, **dictje}

        fc = post_process_results_server_side(result_list,function_properties,extra_properties=EXTRA_PROPERTIES)

        task = ee.batch.Export.table.toCloudStorage(collection=fc,
                                                    description= function_properties["file_name"],
                                                    bucket = GCS_BUCKET,
                                                    fileNamePrefix= GCS_OUTPUT_PATH + function_properties["file_name"],
                                                    fileFormat = "CSV")

        task.start()

        

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014
Index 0.00, Processed: 1 Elapsed: 0:00:02.083631 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I000Y1960M01
Index 1.00, Processed: 2 Elapsed: 0:00:03.664472 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I001Y1960M02
Index 2.00, Processed: 3 Elapsed: 0:00:05.403113 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I002Y1960M03
Index 3.00, Processed: 4 Elapsed: 0:00:07.046550 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_m

Index 35.00, Processed: 36 Elapsed: 0:00:59.834802 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I035Y1962M12
Index 36.00, Processed: 37 Elapsed: 0:01:01.559407 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I036Y1963M01
Index 37.00, Processed: 38 Elapsed: 0:01:03.200681 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I037Y1963M02
Index 38.00, Processed: 39 Elapsed: 0:01:04.910094 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I038Y1963M03
Index 39.00, Processed: 40 Elapsed: 0:01:06.549000 Asset

Index 70.00, Processed: 71 Elapsed: 0:01:55.550884 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I070Y1965M11
Index 71.00, Processed: 72 Elapsed: 0:01:57.127695 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I071Y1965M12
Index 72.00, Processed: 73 Elapsed: 0:01:58.313530 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I072Y1966M01
Index 73.00, Processed: 74 Elapsed: 0:01:59.926988 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I073Y1966M02
Index 74.00, Processed: 75 Elapsed: 0:02:01.578379 Asset

Index 105.00, Processed: 106 Elapsed: 0:02:52.296502 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I105Y1968M10
Index 106.00, Processed: 107 Elapsed: 0:02:54.019383 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I106Y1968M11
Index 107.00, Processed: 108 Elapsed: 0:02:55.650026 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I107Y1968M12
Index 108.00, Processed: 109 Elapsed: 0:02:57.391732 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I108Y1969M01
Index 109.00, Processed: 110 Elapsed: 0:02:59.21

Index 140.00, Processed: 141 Elapsed: 0:03:50.151847 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I140Y1971M09
Index 141.00, Processed: 142 Elapsed: 0:03:51.767964 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I141Y1971M10
Index 142.00, Processed: 143 Elapsed: 0:03:53.384234 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I142Y1971M11
Index 143.00, Processed: 144 Elapsed: 0:03:55.123675 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I143Y1971M12
Index 144.00, Processed: 145 Elapsed: 0:03:56.68

...

Index 560.00, Processed: 561 Elapsed: 0:14:52.354797 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I560Y2006M09
Index 561.00, Processed: 562 Elapsed: 0:14:53.978703 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I561Y2006M10
Index 562.00, Processed: 563 Elapsed: 0:14:55.652384 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I562Y2006M11
Index 563.00, Processed: 564 Elapsed: 0:14:57.370950 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I563Y2006M12
Index 564.00, Processed: 565 Elapsed: 0:14:59.01

Index 595.00, Processed: 596 Elapsed: 0:15:48.415004 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I595Y2009M08
Index 596.00, Processed: 597 Elapsed: 0:15:50.204730 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I596Y2009M09
Index 597.00, Processed: 598 Elapsed: 0:15:51.847537 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I597Y2009M10
Index 598.00, Processed: 599 Elapsed: 0:15:53.532791 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I598Y2009M11
Index 599.00, Processed: 600 Elapsed: 0:15:54.78

Index 630.00, Processed: 631 Elapsed: 0:16:44.382291 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I630Y2012M07
Index 631.00, Processed: 632 Elapsed: 0:16:46.018610 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I631Y2012M08
Index 632.00, Processed: 633 Elapsed: 0:16:47.637678 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I632Y2012M09
Index 633.00, Processed: 634 Elapsed: 0:16:49.266936 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I633Y2012M10
Index 634.00, Processed: 635 Elapsed: 0:16:50.99

Index 5.00, Processed: 666 Elapsed: 0:17:38.130815 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I005Y1965M01
Index 6.00, Processed: 667 Elapsed: 0:17:39.770445 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I006Y1966M01
Index 7.00, Processed: 668 Elapsed: 0:17:41.488414 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I007Y1967M01
Index 8.00, Processed: 669 Elapsed: 0:17:43.096590 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I008Y1968M01
Index 9.00, Processed: 670 Elapsed: 0:17:44.714770 Asset: projec

Index 40.00, Processed: 701 Elapsed: 0:18:35.013047 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I040Y2000M01
Index 41.00, Processed: 702 Elapsed: 0:18:36.676594 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I041Y2001M01
Index 42.00, Processed: 703 Elapsed: 0:18:38.298501 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I042Y2002M01
Index 43.00, Processed: 704 Elapsed: 0:18:40.002953 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I043Y2003M01
Index 44.00, Processed: 705 Elapsed: 0:18:41.123430 Asset: p

In [12]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:19:00.638685


Previous runs:  
1:16:24.123932
